<a href="https://colab.research.google.com/github/emretaylanduman/codes_public/blob/master/SRA_sampler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from urllib.request import Request, urlopen
import re
import math
import textwrap
import os
import random
import time
from nucleus.io import fastq

In [ ]:
dir_hom = 'your_wd' # Set your current wd
os.chdir('/home/etd/Desktop/data_set2')

In [ ]:
cmd_1 = 'fastq-dump -N'
cmd_2 = ' -O ' + dir_hom + ' '

In [ ]:
# Reading downloaded SRR accesion codes.
# These accesions flitered and downloaded by RunSelector for spesific purposes. 

s = 1
file = open('SRR_Acc_Hiseq2000_2.txt','r')  #Downladed accession numbers
a = file.readlines()

In [ ]:
list_sizes = []
# Every file in SRR contains different number of reads.
# This lines controls the number of lines in every SRR accesion codes.
for i in a:

    req = Request('https://trace.ncbi.nlm.nih.gov/Traces/sra/?run='+i)          
    name = i[:-1] + '.fastq'
    response = urlopen(req)
    
    lines = []
    for line in response:
        line=line.decode('utf-8')
        lines.append(line)

    # Searching from the website data file size thats directly converts to the number of reads    
    length = str([x for x in lines if re.search('total_spots', x)])
    multip = str([x for x in lines if re.search('file-size', x)])
    
    
    file_len = length[length.find('spots">')+len('spots">'):length.rfind('<span class')]
    mlyer = multip[multip.find('file-size')+len('file-size '):multip.rfind('</span>bp<')]
    
    # Check if there are Gigabyte or megabyte size file in focused file.
    # Depending on the file size, random number range will be changed

    if mlyer[-1]=='G':
        num_ofl = float(file_len)*math.pow(10.0,6)
    elif mlyer[-1]=='M':
        num_ofl = float(file_len)*math.pow(10.0,6)
    num_ofl = int(num_ofl)
    list_sizes.append(num_ofl)
    x = random.sample(range(1, num_ofl), 100)  # Number of reads that will be taken

    # For every 100 number from range of the file, download reads and process
    for t in x:
        cmd =  cmd_1 +' ' + str(t) + ' -X ' + str(t+1) + cmd_2 + i[:-1]
        cm_d = 'cat *.fastq >> sampled.all'    # Downloaded reads concatanated
        os.system(cmd)
        os.system(cm_d)
        print('Sampled\n')
        if name in os.listdir(dir_hom):        # Check if the file downladed without problem
            print('\nDownloaded ' + i + '.fastq\n')
            print('Processing...\n')
           
            seq_err = []
            qual_err = []
            new_words = []
            new_words2 = []            
            with fastq.FastqReader(name) as reader:
                for record in reader:
                    seq_err = record.sequence
                    qual_err = record.quality
                new_word = ''
                for k in range(len(record.sequence)):
                    location = str(k)
                    new_word = new_word + location + seq_err[k] + qual_err[k] + ' '
                    new_word2 = new_word + seq_err[k] + qual_err[k] + ' '                    
                new_words.append(new_word)
                new_words2.append(new_word2)
            
            mer3_seq = textwrap.wrap(record.sequence,3)
            mer3_qual = textwrap.wrap(record.quality,3)
            with open('sampled_genome_all.txt', 'a+') as f:
                for item in new_word:
                    f.write("%s\n" % item)

            with open('sampled_without_loc.txt', 'a+') as f:
                for item2 in new_word2:
                    f.write("%s\n" % item2)
                print(s)                
#            with open('sampled_5mer_seq.txt', 'a+') as j:
#                for seq in mer5_seq:
#                    if len(seq)==5:
#                        j.write("%s " % seq)
#                j.write("\n")
#            with open('sampled_5mer_qual.txt', 'a+') as k:
#                for qual in mer5_qual:
#                    if len(qual)==5:
#                        k.write("%s " % qual)
#                k.write("\n")
#    
            with open('sampled_3mer_seq.txt', 'a+') as j:
                for seq in mer3_seq:
                    if len(seq)==3:
                        j.write("%s " % seq)
                j.write("\n")
                print('Seq Written\n')
            with open('sampled_3mer_qual.txt', 'a+') as k:
                for qual in mer3_qual:
                    if len(qual)==3:
                        k.write("%s " % qual)
                k.write("\n")
    
                print('Qual Written\n')
                
            s=s+1
            os.remove(name)
            f.close()
            j.close()
            k.close()